In [1]:
# %%
import os
import json
import torch
import ast
import numpy as np
from datetime import UTC
from datetime import datetime as dt
import pandas as pd

from hmc.model import ClassificationModel


In [2]:
base_path = "/home/bruno/storage/data/fma/trains/rock_electronic"
test_csv_path = os.path.join(base_path, 'test.csv')
jobs_path = os.path.join(base_path, 'jobs')
model_id = '20241015_180111'


In [3]:
train_path = os.path.join(jobs_path, model_id)
model_path = os.path.join(train_path, 'best_binary.pth')

torch_path = os.path.join(base_path, 'torch')
metadata_path = os.path.join(base_path, "metadata.json")
labels_path = os.path.join(base_path, "labels.json")
testset_path = os.path.join(torch_path, 'test')

In [4]:
df_test = pd.read_csv(test_csv_path)

In [5]:
# Função para converter a string em lista de np.array
def convert_to_list_of_arrays(val):
    # Converte string para uma lista de strings
    list_str = ast.literal_eval(val)
    # Converte cada item da lista para um array NumPy
    return [np.array(item) for item in list_str]


In [6]:
# Aplicar ast.literal_eval para converter as strings em listas
df_test['all_binarized'] = df_test.all_binarized.apply(convert_to_list_of_arrays)

In [ ]:
df_test

,track_genres_all,track_id,file_path,index,track_genre_top,y_true,level1,level2,level3,level4,all_binarized
0,"[66, 12]",33038,/mnt/disks/data/fma/fma_large/033/033038.mp3,21041,Rock,"[[12, 66]]","[1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0]","[[1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
1,"[183, 286, 15]",98938,/mnt/disks/data/fma/fma_large/098/098938.mp3,66294,Electronic,"[[15, 183], [15, 286]]","[0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0]","[[0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,[15],6775,/mnt/disks/data/fma/fma_large/006/006775.mp3,3560,Electronic,[[15]],"[0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0]","[[0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[181, 15]",63354,/mnt/disks/data/fma/fma_large/063/063354.mp3,42675,Electronic,"[[15, 181]]","[0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0]","[[0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[25, 12, 109]",67165,/mnt/disks/data/fma/fma_large/067/067165.mp3,45761,Rock,"[[12, 25, 109]]","[1, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0]","[[1, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...
4809,[12],3751,/mnt/disks/data/fma/fma_large/003/003751.mp3,2052,Rock,[[12]],"[1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0]","[[1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4810,"[183, 495, 15]",99310,/mnt/disks/data/fma/fma_large/099/099310.mp3,66578,Electronic,"[[15, 183], [15, 495]]","[0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0]","[[0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4811,"[25, 12, 109, 31]",75793,/mnt/disks/data/fma/fma_large/075/075793.mp3,51944,Rock,"[[12, 25, 109]]","[1, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0]","[[1, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4812,"[27, 12, 359]",51360,/mnt/disks/data/fma/fma_large/051/051360.mp3,33790,Rock,"[[12, 27], [12, 359]]","[1, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0]","[[1, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [8]:
df_test.level2[4809]

'[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]'

In [ ]:
with open(metadata_path, 'r') as f:
    metadata = json.loads(f.read())
    print(metadata)

with open(labels_path, 'r') as f:
    labels = json.loads(f.read())


{'sequence_size': 1280, 'max_depth': 4, 'levels_size': [2, 29, 15, 2], 'val_path': '/home/bruno/storage/data/fma/trains/rock_electronic/tfrecrods/val', 'train_path': '/home/bruno/storage/data/fma/trains/rock_electronic/tfrecrods/train', 'test_path': '/home/bruno/storage/data/fma/trains/rock_electronic/tfrecrods/test', 'val_torch_path': '/home/bruno/storage/data/fma/trains/rock_electronic/torch/val', 'train_torch_path': '/home/bruno/storage/data/fma/trains/rock_electronic/torch/train', 'test_torch_path': '/home/bruno/storage/data/fma/trains/rock_electronic/torch/test', 'val_csv': '/home/bruno/storage/data/fma/trains/rock_electronic/val.csv', 'train_csv': '/home/bruno/storage/data/fma/trains/rock_electronic/train.csv', 'test_csv': '/home/bruno/storage/data/fma/trains/rock_electronic/test.csv', 'trainset_count': 19030, 'validationset_count': 2007, 'testset_count': 4814}


In [10]:
params = {
    'levels_size': labels['levels_size'],
    'sequence_size': metadata['sequence_size'],
    'dropouts': [0.3, 0.3, 0.3, 0.3],
    'thresholds': [0.5, 0.3, 0.3, 0.3]
}


In [11]:

model = ClassificationModel(**params)

# Carregar o state_dict
state_dict = torch.load(model_path)

# Imprimir as chaves do state_dict
print("Chaves do state_dict salvo:")
print(state_dict.keys())

# Imprimir as chaves do modelo
print("Chaves do modelo definido:")
print(model.state_dict().keys())

#model_load = torch.load(binary_model)
model.load_state_dict(state_dict)

/tmp/ipykernel_793037/195874492.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(model_path)


Chaves do state_dict salvo:
odict_keys(['levels.0.fc1.weight', 'levels.0.fc1.bias', 'levels.0.fc2.weight', 'levels.0.fc2.bias', 'levels.1.fc1.weight', 'levels.1.fc1.bias', 'levels.1.fc2.weight', 'levels.1.fc2.bias', 'levels.2.fc1.weight', 'levels.2.fc1.bias', 'levels.2.fc2.weight', 'levels.2.fc2.bias', 'levels.3.fc1.weight', 'levels.3.fc1.bias', 'levels.3.fc2.weight', 'levels.3.fc2.bias'])
Chaves do modelo definido:
odict_keys(['levels.0.fc1.weight', 'levels.0.fc1.bias', 'levels.0.fc2.weight', 'levels.0.fc2.bias', 'levels.1.fc1.weight', 'levels.1.fc1.bias', 'levels.1.fc2.weight', 'levels.1.fc2.bias', 'levels.2.fc1.weight', 'levels.2.fc1.bias', 'levels.2.fc2.weight', 'levels.2.fc2.bias', 'levels.3.fc1.weight', 'levels.3.fc1.bias', 'levels.3.fc2.weight', 'levels.3.fc2.bias'])


<All keys matched successfully>

In [12]:
model.to('cuda')

ClassificationModel(
  (levels): ModuleList(
    (0): BuildClassification(
      (fc1): Linear(in_features=1280, out_features=640, bias=True)
      (relu1): ReLU()
      (dropout1): Dropout(p=0.3, inplace=False)
      (fc2): Linear(in_features=640, out_features=2, bias=True)
    )
    (1): BuildClassification(
      (fc1): Linear(in_features=1280, out_features=640, bias=True)
      (relu1): ReLU()
      (dropout1): Dropout(p=0.3, inplace=False)
      (fc2): Linear(in_features=640, out_features=29, bias=True)
    )
    (2): BuildClassification(
      (fc1): Linear(in_features=1280, out_features=640, bias=True)
      (relu1): ReLU()
      (dropout1): Dropout(p=0.3, inplace=False)
      (fc2): Linear(in_features=640, out_features=15, bias=True)
    )
    (3): BuildClassification(
      (fc1): Linear(in_features=1280, out_features=640, bias=True)
      (relu1): ReLU()
      (dropout1): Dropout(p=0.3, inplace=False)
      (fc2): Linear(in_features=640, out_features=2, bias=True)
    )
  )
)

In [13]:
df_test

,track_genres_all,track_id,file_path,index,track_genre_top,y_true,level1,level2,level3,level4,all_binarized
0,"[66, 12]",33038,/mnt/disks/data/fma/fma_large/033/033038.mp3,21041,Rock,"[[12, 66]]","[1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0]","[[1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, ..."
1,"[183, 286, 15]",98938,/mnt/disks/data/fma/fma_large/098/098938.mp3,66294,Electronic,"[[15, 183], [15, 286]]","[0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0]","[[0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,[15],6775,/mnt/disks/data/fma/fma_large/006/006775.mp3,3560,Electronic,[[15]],"[0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0]","[[0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[181, 15]",63354,/mnt/disks/data/fma/fma_large/063/063354.mp3,42675,Electronic,"[[15, 181]]","[0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0]","[[0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[25, 12, 109]",67165,/mnt/disks/data/fma/fma_large/067/067165.mp3,45761,Rock,"[[12, 25, 109]]","[1, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0]","[[1, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...,...,...,...
4809,[12],3751,/mnt/disks/data/fma/fma_large/003/003751.mp3,2052,Rock,[[12]],"[1, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0]","[[1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4810,"[183, 495, 15]",99310,/mnt/disks/data/fma/fma_large/099/099310.mp3,66578,Electronic,"[[15, 183], [15, 495]]","[0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0]","[[0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4811,"[25, 12, 109, 31]",75793,/mnt/disks/data/fma/fma_large/075/075793.mp3,51944,Rock,"[[12, 25, 109]]","[1, 0]","[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0]","[[1, 0], [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4812,"[27, 12, 359]",51360,/mnt/disks/data/fma/fma_large/051/051360.mp3,33790,Rock,"[[12, 27], [12, 359]]","[1, 0]","[0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]","[0, 0]","[[1, 0], [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [14]:
# %%
# Salvar os argumentos atuais
time_start = dt.now(UTC)
print("[{}] Predict started at {}".format(id, time_start.strftime("%H:%M:%S")))
print(".......................................")

track_id, predict = model.predict(base_path, batch_size=64)

time_end = dt.now(UTC)
time_elapsed = time_end - time_start
print(".......................................")
print("[{}] Predict finished at {} / elapsed time {}s".format(id, time_end.strftime("%H:%M:%S"), time_elapsed.total_seconds()))

[<built-in function id>] Predict started at 21:03:11
.......................................
.......................................
[<built-in function id>] Predict finished at 21:03:12 / elapsed time 0.302548s


In [15]:
df_test.level1[0]

'[1, 0]'

In [18]:
predict['level1'][0].argmax()

tensor([ 2.6344, -2.6541], device='cuda:0')

In [17]:
len(predict)

4

In [36]:
examples_pred = []
levels_pred = {}
for level, pred in enumerate(predict, start=1):
    level_name = f'level{level}'
    levels_pred[level_name] = pred

In [37]:
levels_pred['level2']

'level2'